# Common imports

In [ ]:
%matplotlib inline
import os
import sys
import glob
import re
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from PIL import Image

# Provided functions
### Image loading and saving

In [ ]:
def LoadFaceImages(pathname, subject_name, num_images):
    """
    Load the set of face images.  
    The routine returns
        ambimage: image illuminated under the ambient lighting
        imarray: a 3-D array of images, h x w x Nimages
        lightdirs: Nimages x 3 array of light source directions
    """

    def load_image(fname):
        return np.asarray(Image.open(fname))

    def fname_to_ang(fname):
        yale_name = os.path.basename(fname)
        return int(yale_name[12:16]), int(yale_name[17:20])

    def sph2cart(az, el, r):
        rcos_theta = r * np.cos(el)
        x = rcos_theta * np.cos(az)
        y = rcos_theta * np.sin(az)
        z = r * np.sin(el)
        return x, y, z

    ambimage = load_image(
        os.path.join(pathname, subject_name + '_P00_Ambient.pgm'))
    im_list = glob.glob(os.path.join(pathname, subject_name + '_P00A*.pgm'))
    if num_images <= len(im_list):
        im_sub_list = np.random.choice(im_list, num_images, replace=False)
    else:
        print(
            'Total available images is less than specified.\nProceeding with %d images.\n'
            % len(im_list))
        im_sub_list = im_list
    im_sub_list.sort()
    imarray = np.stack([load_image(fname) for fname in im_sub_list], axis=-1)
    Ang = np.array([fname_to_ang(fname) for fname in im_sub_list])

    x, y, z = sph2cart(Ang[:, 0] / 180.0 * np.pi, Ang[:, 1] / 180.0 * np.pi, 1)
    lightdirs = np.stack([y, z, x], axis=-1)
    return ambimage, imarray, lightdirs

In [ ]:
def save_outputs(subject_name, albedo_image, surface_normals):
    im = Image.fromarray((albedo_image*255).astype(np.uint8))
    im.save("%s_albedo.jpg" % subject_name)
    im = Image.fromarray((surface_normals[:,:,0]*128+128).astype(np.uint8))
    im.save("%s_normals_x.jpg" % subject_name)
    im = Image.fromarray((surface_normals[:,:,1]*128+128).astype(np.uint8))
    im.save("%s_normals_y.jpg" % subject_name)
    im = Image.fromarray((surface_normals[:,:,2]*128+128).astype(np.uint8))
    im.save("%s_normals_z.jpg" % subject_name)

### Plot the height map

In [ ]:
def set_aspect_equal_3d(ax):
    """https://stackoverflow.com/questions/13685386"""
    """Fix equal aspect bug for 3D plots."""
    xlim = ax.get_xlim3d()
    ylim = ax.get_ylim3d()
    zlim = ax.get_zlim3d()
    from numpy import mean
    xmean = mean(xlim)
    ymean = mean(ylim)
    zmean = mean(zlim)
    plot_radius = max([
        abs(lim - mean_)
        for lims, mean_ in ((xlim, xmean), (ylim, ymean), (zlim, zmean))
        for lim in lims
    ])
    ax.set_xlim3d([xmean - plot_radius, xmean + plot_radius])
    ax.set_ylim3d([ymean - plot_radius, ymean + plot_radius])
    ax.set_zlim3d([zmean - plot_radius, zmean + plot_radius])


def display_output(albedo_image, height_map, azim=20, elev=20):
    fig = plt.figure()
    plt.imshow(albedo_image, cmap='gray')
    plt.axis('off')
    
    fig = plt.figure(figsize=(10, 10))
    ax = plt.axes(projection='3d') # NOTE gca() with kwargs is deprecated
    ax.view_init(azim=azim, elev=elev)
    X = np.arange(albedo_image.shape[0])
    Y = np.arange(albedo_image.shape[1])
    X, Y = np.meshgrid(Y, X)
    H = np.flipud(np.fliplr(height_map))
    A = np.flipud(np.fliplr(albedo_image))
    A = np.stack([A, A, A], axis=-1)
    ax.xaxis.set_ticks([])
    ax.xaxis.set_label_text('Z')
    ax.yaxis.set_ticks([])
    ax.yaxis.set_label_text('X')
    ax.zaxis.set_ticks([])
    ax.yaxis.set_label_text('Y')
    surf = ax.plot_surface(
        H, X, Y, cmap='gray', facecolors=A, linewidth=0, antialiased=False)
    set_aspect_equal_3d(ax)

### Plot the surface norms. 

In [ ]:
def plot_surface_normals(surface_normals):
    """
    surface_normals: h x w x 3 matrix.
    """
    fig = plt.figure()
    ax = plt.subplot(1, 3, 1)
    ax.axis('off')
    ax.set_title('X')
    im = ax.imshow(surface_normals[:,:,0])
    ax = plt.subplot(1, 3, 2)
    ax.axis('off')
    ax.set_title('Y')
    im = ax.imshow(surface_normals[:,:,1])
    ax = plt.subplot(1, 3, 3)
    ax.axis('off')
    ax.set_title('Z')
    im = ax.imshow(surface_normals[:,:,2])

# Your implementation

In [ ]:
def preprocess(ambimage, imarray):
    """
    preprocess the data: 
        1. subtract ambient_image from each image in imarray.
        2. make sure no pixel is less than zero.
        3. rescale values in imarray to be between 0 and 1.
    Inputs:
        ambimage: h x w
        imarray: h x w x Nimages
    Outputs:
        processed_imarray: h x w x Nimages
    """
    processed_imarray = imarray.astype(float)
    # NOTE
    #   n_images axis should be the slow axis (0) for better performance
    processed_imarray -= ambimage[..., np.newaxis]

    # 0-clipping, normalize to [0, 1]
    processed_imarray = np.clip(processed_imarray, 0, None)
    processed_imarray /= np.max(processed_imarray)

    return processed_imarray

In [ ]:
def photometric_stereo(imarray, light_dirs):
    """
    Inputs:
        imarray:  h x w x Nimages
        light_dirs: Nimages x 3
    Outputs:
        albedo_image: h x w
        surface_norms: h x w x 3
    """
    # flatten each image
    h, w, n_images = imarray.shape
    imarray = np.reshape(imarray, (-1, n_images))
    # NOTE
    #   since sample dimension is at axis -1, this become y^T = Ax
    g, residuals, _, _ = np.linalg.lstsq(light_dirs, imarray.T, rcond=None)
    r2 = 1 - residuals / (n_images * imarray.var(axis=-1))
    print(f'Median of r^2 during LS fit is {np.median(r2):.4f}')

    # rho = ||g||_2
    # NOTE
    #   ord=None is 2-norm
    albedo_image = np.linalg.norm(g, ord=None, axis=0, keepdims=False)
    albedo_image = np.reshape(albedo_image, (h, w))

    # N = g / rho
    g = np.reshape(g.T, (h, w, -1))
    surface_normals = g / albedo_image[..., np.newaxis]
    
    return albedo_image, surface_normals

In [ ]:
def get_surface(surface_normals, integration_method, **kwargs):
    """
    Inputs:
        surface_normals:h x w x 3
        integration_method: string in ['average', 'column', 'row', 'random']
    Outputs:
        height_map: h x w
    """
    f_x = surface_normals[..., 0] / surface_normals[..., 2]
    f_y = surface_normals[..., 1] / surface_normals[..., 2]
    
    def column(**kwargs):
        """column -> row"""
        height_map = np.zeros_like(f_x)
        height_map[0, :] = np.cumsum(f_x[0, :])
        height_map[1:, :] = f_y[1:, :]
        height_map = np.cumsum(height_map, axis=0)
        return height_map

    def row(**kwargs):
        """row -> column"""
        height_map = np.zeros_like(f_y)
        height_map[:, 0] = np.cumsum(f_y[:, 0])
        height_map[:, 1:] = f_x[:, 1:]
        height_map = np.cumsum(height_map, axis=1)
        return height_map 
    
    def average(**kwargs):
        return (column() + row()) / 2 

    def random(n_random=5, **kwargs):
        """random walk, column takes precedence"""
        height_map = np.zeros_like(f_x)
        ny, nx = height_map.shape
        for y in range(ny):
            for x in range(nx):
                # steps to the origin, 1-hot vector, 1 moves along x, 0 moves along y
                steps = np.array([1]*x + [0]*y, dtype=int)

                acc = 0
                # random walk multiple times
                for n in range(n_random):
                    # starts with column
                    acc += f_x[0, 0] 
                    # in-place shuffle only occurs on first dimension
                    np.random.shuffle(steps)

                    # go over the trace and sum f_x/f_y accordingly
                    dx, dy = 0, 0
                    for s in steps:
                        if s:
                            dx += 1
                            acc += f_x[dy, dx]
                        else:
                            dy += 1
                            acc += f_y[dy, dx]
                # average out
                acc /= n_random

                # write back
                height_map[y, x] = acc
        return height_map

    def frankotchellappa(**kwargs):
        from numpy.fft import fft2, ifft2, fftfreq

        ny, nx = f_x.shape
        k_x, k_y = np.meshgrid(
            fftfreq(nx) * 2*np.pi,
            fftfreq(ny) * 2*np.pi, 
            indexing='xy'
        )

        num = -1j * k_x * fft2(f_x) - 1j * k_y * fft2(f_y)
        den = k_x**2 + k_y**2 + np.finfo(float).eps # prevent div-0

        height_map = ifft2(num / den)
        height_map -= np.mean(np.real(height_map))
        return height_map

    try:
        return {
            'random': random, 
            'column': column, 
            'row': row, 
            'average': average,
            'frankotchellappa': frankotchellappa
        }[integration_method](**kwargs)
    except KeyError:
        raise ValueError(f"unknown integration method '{integration_method}'")

# Main function

In [ ]:
class StopExecution(Exception):
    """Stop current and subsequent cells quietly."""
    def _render_traceback_(self):
        pass

In [ ]:
root_path = 'croppedyale/'
subject_name = 'yaleB07'
integration_method = 'average'
save_flag = False

full_path = '%s%s' % (root_path, subject_name)
ambient_image, imarray, light_dirs = LoadFaceImages(full_path, subject_name,
                                                    64)

processed_imarray = preprocess(ambient_image, imarray)

albedo_image, surface_normals = photometric_stereo(processed_imarray,
                                                   light_dirs)

height_map = get_surface(surface_normals, integration_method, n_random=5)

if save_flag:
    save_outputs(subject_name, albedo_image, surface_normals)

In [ ]:
plot_surface_normals(surface_normals)

In [ ]:
for i in range(64):
    im = Image.fromarray(imarray[..., i].astype(np.uint8))
    im.save(f"{subject_name}_{i:02d}.jpg")

In [ ]:
display_output(albedo_image, height_map, azim=-60, elev=30)
plt.savefig(
    f'{subject_name}_{integration_method}_az{azim}_el{elev}_fs.png', 
    bbox_inches='tight'
)

In [ ]:
raise StopExecution

In [ ]:
# part 2.1.1 
#   save height map from different integration method
root_path = 'croppedyale/'
subject_name = 'yaleB07'

for integration_method in ('column', 'row', 'average', 'random'):
    full_path = '%s%s' % (root_path, subject_name)
    ambient_image, imarray, light_dirs = LoadFaceImages(full_path, subject_name,
                                                        64)

    processed_imarray = preprocess(ambient_image, imarray)

    albedo_image, surface_normals = photometric_stereo(processed_imarray,
                                                    light_dirs)

    height_map = get_surface(surface_normals, integration_method, n_random=5)

    for azim in (-60, 0, 60):
        for elev in (-30, 0, 30):
            display_output(albedo_image, height_map, azim=azim, elev=elev)
            plt.savefig(
                f'{subject_name}_{integration_method}_az{azim}_el{elev}.png', bbox_inches='tight'
            )

In [ ]:
# part 2.1.2 
#   save height map from different integration method
root_path = 'croppedyale/'
integration_method = 'frankotchellappa'# 'random'

for subject_name in ('yaleB05', ): #('yaleB01', 'yaleB02', 'yaleB05', 'yaleB07'):
    full_path = '%s%s' % (root_path, subject_name)
    ambient_image, imarray, light_dirs = LoadFaceImages(full_path, subject_name,
                                                        64)

    processed_imarray = preprocess(ambient_image, imarray)

    albedo_image, surface_normals = photometric_stereo(processed_imarray,
                                                    light_dirs)

    height_map = get_surface(surface_normals, integration_method, n_random=5)

    for azim in (-60, 60):
        for elev in (-30, 30):
            display_output(albedo_image, height_map, azim=azim, elev=elev)
            plt.savefig(
                f'{subject_name}_{integration_method}_az{azim}_el{elev}.png', bbox_inches='tight'
            )

In [ ]:
# part 2.3 
#   execution time
import time

root_path = 'croppedyale/'
subject_name = 'yaleB07'
integration_method = 'random'

n_repeat = 10

for integration_method in ('column', 'row', 'average'):
    full_path = '%s%s' % (root_path, subject_name)
    ambient_image, imarray, light_dirs = LoadFaceImages(full_path, subject_name,
                                                        64)

    processed_imarray = preprocess(ambient_image, imarray)

    albedo_image, surface_normals = photometric_stereo(processed_imarray,
                                                    light_dirs)


    start_time = time.time()
    for _ in range(n_repeat):
        height_map = get_surface(surface_normals, integration_method, n_random=5)
    elapsed_time = time.time() - start_time
    average_time = elapsed_time / n_repeat
    
    print(f'{integration_method}, {average_time}')

In [ ]:
# 4 subset

remove_index = [
    3, 
    30, 31, 32, 33, 34, 
    50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63
]

root_path = 'croppedyale/'
subject_name = 'yaleB07'
integration_method = 'average'
save_flag = False

full_path = '%s%s' % (root_path, subject_name)
ambient_image, imarray, light_dirs = LoadFaceImages(full_path, subject_name, 64)

# remove 
imarray = np.stack(
    [imarray[..., i] for i in range(64) if i not in remove_index], axis=-1
)
light_dirs = np.array([light_dirs[i, ...] for i in range(64) if i not in remove_index])

processed_imarray = preprocess(ambient_image, imarray)

albedo_image_subset, surface_normals = photometric_stereo(processed_imarray,
                                                   light_dirs)

height_map_subset = get_surface(surface_normals, integration_method, n_random=5)

In [ ]:
display_output(albedo_image, height_map, azim=60, elev=10)
plt.savefig(
    f'{subject_name}_{integration_method}_ref.png', bbox_inches='tight'
)
display_output(albedo_image_subset, height_map_subset, azim=60, elev=10)
plt.savefig(
    f'{subject_name}_{integration_method}_subset.png', bbox_inches='tight'
)

In [ ]:
# save specific view
display_output(albedo_image, height_map, azim=300, elev=30)
plt.savefig(f'{subject_name}_{integration_method}.png', bbox_inches='tight')